In [1]:
# AI MVP Project from datatec.studio
!pip install transformers torch accelerate bitsandbytes langchain
!pip install -U sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.llms import HuggingFacePipeline

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [3]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", load_in_4bit=True, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
import transformers
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=1000
)

In [7]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [8]:
dummy_company_profile = """
Company Profile:

Company Name: Dummy-Gpt2-Datatec-Studio Inc
At Dummy-Gpt2-Datatec-Studio Inc, we are a cutting-edge technology company committed to transforming visionary ideas into tangible realities. Our mission is to pioneer innovation that makes a meaningful impact on the world, from extending human longevity to addressing critical environmental challenges and revolutionizing the way we travel.

Portfolio:
Life-Enhancing Pharmaceuticals: Our portfolio includes groundbreaking pharmaceutical research aimed at developing vaccines and treatments to prevent diseases, potentially adding an additional 20 years to the human lifespan.
Environmental Sustainability: We are leaders in developing innovative solutions to combat climate change and ensure a sustainable future for the planet, encompassing green energy, waste reduction, and carbon footprint mitigation.
Advanced Transportation: In our transportation division, we engineer state-of-the-art terrestrial transportation solutions, promising ultra-fast and efficient modes of travel that will reshape the way we move.

Products:
Green Energy Devices: Our product line features advanced devices that empower individuals and businesses to generate clean and renewable energy, rendering traditional power plants and fossil fuels obsolete.
Space Exploration Services: Our space exploration services offer accessible and safe travel options to explore colonies on celestial bodies such as solar planets and planetary moons, opening new frontiers for exploration.
Quantum Computing Solutions: We provide cutting-edge quantum computing solutions, exponentially boosting computational power and network speeds, thus ushering in a new era of technological advancement.
Holographic Communication: Our holographic communication technology replaces traditional televisions and phones with immersive, interactive experiences. Integrating the senses of touch and smell, we bring the virtual world closer to reality.

Services:
Genetic Engineering: Our genetic engineering services explore revolutionary concepts like Wi-Fi-emitting plants, potentially transforming the way we connect and communicate in a lush and connected world.
Futuristic Planning Consultation: We specialize in assisting individuals and organizations in planning for the distant future. Leveraging transformative technologies, we help shape and anticipate possible outcomes to achieve long-term goals.

Contact:
For inquiries and collaborations, please reach out to us at:
Email: contact@dummygpt2datatecstudio.com
Phone: +1-123-456-7890
Address: 123 Innovator's Lane, Futuroville, Earth

Community Engagement:
At Dummy-Gpt2-Datatec-Studio Inc, we take our commitment to community involvement seriously. We actively support STEM education programs, empowering future generations to be at the forefront of scientific and technological advancements that will shape our collective future.
"""

In [9]:
# Create Document object from text documents
docs = [Document(page_content=post) for post in [dummy_company_profile]]

# Split documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=10, separators=['\n\n', '\n', '.']
)

document_chunks = text_splitter.split_documents(docs)

In [10]:
embedding_model = SentenceTransformerEmbeddings(model_name='BAAI/bge-large-en-v1.5')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [11]:
chroma_db = Chroma.from_documents(document_chunks, embedding_model)

retriever = chroma_db.as_retriever()

In [12]:
qa_chain = RetrievalQA.from_chain_type(mistral_llm, retriever=retriever)

In [13]:
question = input("Please enter your question: ")

response = qa_chain({"query": question})

print(response['result'])

Please enter your question: What are your products?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or tra

 Dummy-Gpt2-Datatec-Studio Inc offers various products, including Green Energy Devices, Space Exploration Services, Quantum Computing Solutions, and Holographic Communication.
